In [ ]:
import numpy as np
import cv2

In [ ]:
# Specify filepaths here
input_path = ""
output_path = ""

In [ ]:
frames = []

# Read input video
cap = cv2.VideoCapture(input_path)

# Get frame count
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) 

fps = cap.get(cv2.CAP_PROP_FPS)
 
# Get width and height of video stream
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
 
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
 
out = cv2.VideoWriter(output_path, fourcc, fps, w, h)

In [ ]:
# Read first frame
_, prev = cap.read()
 
# Convert frame to grayscale
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

mask = np.zeros_like(prev)

In [ ]:
# Pre-define transformation-store array
transforms = np.zeros((n_frames-1, 3), np.float32) 

prev_pts = cv2.goodFeaturesToTrack(prev_gray,
                                     maxCorners=200,
                                     qualityLevel=0.01,
                                     minDistance=30,
                                     blockSize=3)

In [ ]:
OF_points_old = []
OF_points_new = []
for i in range(n_frames-2):
    
    success, curr = cap.read()
    
    if not success: 
        break 
        
    # Converting from BGR to grayscale
    curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
    
    curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None) 

    # Sanity check
    assert prev_pts.shape == curr_pts.shape 

    # Filter only valid points
    idx = np.where(status==1)[0]
    good_old = prev_pts[idx]
    good_new = curr_pts[idx]
   
    # Find transformation matrix
    m = cv2. estimateAffine2D(good_old, good_new)

    OF_points_old.append(good_old)
    OF_points_new.append(good_new)
  
    # Extract traslations on x and y axes
    dx = m[0][0][0]
    dy = m[0][1][0]
 
    # Extract rotation angle
    dangle = np.arctan2(m[0][1][0], m[0][0][0])
 
    # Store transformation
    transforms[i] = [dx, dy, dangle]

    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), (0, 0, 255), 1)
        curr = cv2.circle(curr, (int(a), int(b)), 5, (0, 0, 255), -1)


    img = cv2.add(curr,mask)
    frames.append(img)

    # Now update  previous frame and previous points
    prev_pts = good_new.reshape(-1, 1, 2)
    prev_gray = curr_gray.copy()

    print("Frame: " + str(i) +  "/" + str(n_frames) + " -  Tracked points : " + str(len(prev_pts)))
    out.write(img)
    
# Release video
cap.release()
out.release()

# Close windows
cv2.destroyAllWindows()